In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal,norm
import math
from hmmlearn import hmm
import numpy.random as npr
from pyslds.models import DefaultSLDS
from pylds.util import random_rotation


/Users/shenwang/miniconda3/envs/ml/lib/python3.12/site-packages/pybasicbayes/distributions/multinomial.py:21: UserWarning: using slow sample_crp_tablecounts
  warn('using slow sample_crp_tablecounts')
/Users/shenwang/miniconda3/envs/ml/lib/python3.12/site-packages/pybasicbayes/distributions/negativebinomial.py:25: UserWarning: using slow sample_crp_tablecounts
  warn('using slow sample_crp_tablecounts')


In [2]:
class HMM(object):
    def __init__(self,K = None, T=None, transition_dis = None, init_prob = None, emission = None, y=None):
        
        self.y = y # T x N
        self.K = K # dimension of hidden discrete states
        self.T = T # number of observations

        self.transition = transition_dis # the discrete variable transition probability matrix, K x K
        self.init_prob = init_prob # the initial probability of the discrete variable, K
        
        self.emission = emission # this will be the effective emission calculated for mean-field method, T x K
    
    def forward_backward(self):
        forward_hat = np.zeros((self.T,self.K))
        backward_hat = np.zeros((self.T,self.K))
        scale_factors = np.zeros((self.T))

        for i in range(self.K):
            forward_hat[0,i] = self.init_prob[i] * self.emission[0,i]
        scale_factors[0] = np.sum(forward_hat[0,:])
        forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
        
        for t in range(self.T-1):
            for i in range(self.K):
                for j in range(self.K):
                    forward_hat[t+1,i] += forward_hat[t,j] * self.transition[j,i]
                forward_hat[t+1,i] = forward_hat[t+1,i] * self.emission[t,i]
            scale_factors[t+1] = np.sum(forward_hat[t+1,:])
            forward_hat[t+1,:] = forward_hat[t+1,:]/scale_factors[t+1]

        backward_hat[-1,:] = scale_factors[-1]
        for t in reversed(range(self.T-1)):
            for i in range(self.K):
                for j in range(self.K):
                    backward_hat[t,i] += backward_hat[t+1,j] * self.emission[t+1,j] * self.transition[i,j]
            backward_hat[t,:] = backward_hat[t,:]/scale_factors[t]

        a = np.zeros((self.T,self.K))
        b = np.zeros((self.T,self.K,self.K))
        for i in range(self.T):
            for j in range(self.K):
                a[i,j] = forward_hat[i,j]*backward_hat[i,j]
            temp = np.sum(a[i,:])
            a[i,:] = a[i,:]/temp

        for t in range(self.T-1):
            for i in range(self.K):
                for j in range(self.K):
                    b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j] * self.transition[i,j] * self.emission[t,j]

        self.gamma = a
        self.xi = b
        self.forward_hat = forward_hat
        self.backward_hat = backward_hat
        
    def generate_states(self):
        model = hmm.CategoricalHMM(n_components=self.K)
        model.emissionprob_ = np.array([[1,0],[0,1]])
        model.transmat_ = self.transition
        model.startprob_ = self.init_prob
        data,states = model.sample(n_samples = self.T)

        return states


In [3]:
class timeVaryingLDS(object):
    def __init__(self,T=None, transition_con = None, emission = None, Q = None, R = None, u0 = None, V0 = None, y=None):
        
        self.y = y # T x N
        self.M = transition_con.shape[1] # dimension of hidden continuous states
        self.T = T # number of observations
        self.N = self.y.shape[1] # dimension of the observations
        
        self.C = emission # the emission probability of the continuous variable, T x N x M
        self.A = transition_con # the continuous variable transition probability matrix, T x M x M 
        
        self.Q = Q # Q is the covariance matrix of noise term added to the hidden continuous state transition, T x M x M
        self.R = R # R is the covariance matrix of noise term added to the emission, T x N x N
        
        self.u0 = u0 # u0 is the initial estimate of the mean of x1, M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of x1, M x M

        self.P = np.zeros((self.T, self.M, self.M))
        self.P[:,:,:,] = np.eye(self.M)  # P is an intermediate variable during inference, T x M x M
        self.u = np.zeros((self.T, self.M)) # T x M x 1
        self.V = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.K = np.zeros((self.T, self.M, self.N)) # T x M x N
        self.c = np.zeros((self.T)) # T x 1

        # for backward passing
        self.u_hat = np.zeros((self.T, self.M)) # T x M x 1
        self.V_hat = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.J = np.zeros((self.T, self.M, self.M)) # T x M x M


    
    def kalman_filtering(self):
        S_temp = np.matmul(np.matmul(self.C[0], self.V0), self.C[0].T) + self.R[0]
        Q_temp = np.matmul(self.C[0], self.u0)
        I = np.eye(self.M)
        
        self.V[0] = np.matmul((I - np.matmul(np.matmul(np.matmul(self.V0, self.C[0].T), np.linalg.inv(S_temp)), self.C[0])), self.V0)
        self.P[0] = np.matmul(np.matmul(self.A[0], self.V[0]), self.A[0].T) + self.Q[0]
        self.K[0] = np.matmul(np.matmul(self.P[0], self.C[0].T), np.linalg.inv(np.matmul(np.matmul(self.C[0], self.P[0]), self.C[0].T) + self.R[0]))
        self.u[0] = self.u0 + np.matmul(self.K[0], self.y[0] - Q_temp)

        self.c[0] = multivariate_normal.pdf(self.y[0], Q_temp, S_temp)

        for i in range(1,self.T,1):
            I = np.eye(self.M)
            Q_temp = np.matmul(np.matmul(self.C[i], self.A[i]), self.u[i-1])
            
            self.V[i] = np.matmul((I - np.matmul(self.K[i-1], self.C[i])), self.P[i-1])
            self.P[i] = np.matmul(np.matmul(self.A[i], self.V[i]), self.A[i].T) + self.Q[i]
            S_temp = np.matmul(np.matmul(self.C[i], self.P[i]), self.C[i].T) + self.R[i]
            # print('C[i] is',self.C[i],'R[i] is',self.R[i],'A[i] is',self.A[i],'Q[i] is',self.Q[i],'P[i] is',self.P[i],'V[i] is',self.V[i])
            self.K[i] = np.matmul(np.matmul(self.P[i], self.C[i].T), np.linalg.inv(S_temp))

            self.u[i] = np.matmul(self.A[i], self.u[i-1]) + np.matmul(self.K[i-1], self.y[i] - Q_temp)
            # print('i is:',i,'s_temp is:',S_temp)
            self.c[i] = multivariate_normal.pdf(self.y[i], Q_temp, S_temp)

    def kalman_smoothing(self):

        self.u_hat[-1] = self.u[-1]
        self.V_hat[-1] = self.V[-1]

        for i in range(self.T-2,-1,-1):
            # print(i,self.V[i],self.A[i].T,self.P[i])
            self.J[i] = np.matmul(np.matmul(self.V[i], self.A[i].T), np.linalg.inv(self.P[i]))
            self.u_hat[i] = self.u[i] + np.matmul(self.J[i], self.u_hat[i+1] - np.matmul(self.A[i], self.u[i]))
            self.V_hat[i] = self.V[i] + np.matmul(np.matmul(self.J[i], self.V_hat[i+1] - self.P[i]), self.J[i].T)
    
    def kalman_learning(self):
        self.u0 = self.u_hat[0]
        self.V0 = self.V_hat[0] + np.outer(self.u_hat[0], self.u_hat[0].T) - np.outer(self.u_hat[0], self.u_hat[0].T)

        # E[z[n]] : M x 1
        # E[z[n]z[n-1].T] : M x M
        # E[z[n]z[n].T] : M x M

        self.XtXt_1 = np.zeros((self.T,self.M,self.M))
        self.XtXt = np.zeros((self.T,self.M,self.M))
        self.Xt_1Xt = np.zeros((self.T,self.M,self.M))

        self.YtXt = np.zeros((self.T,self.N,self.M))
        self.YtYt = np.zeros((self.T,self.N,self.N))
        self.XtYt = np.zeros((self.T,self.M,self.N))
        
        self.XtXt[0] += self.V_hat[0] + np.outer(self.u_hat[0], self.u_hat[0].T)
        
        for i in range(1,self.T,1):
            
            self.XtXt_1[i] = np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T) # z[n]z[n-1]
            
            self.XtXt[i] = self.V_hat[i] + np.outer(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            self.Xt_1Xt[i] = (np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T)).T #z[n-1]z[n]

        for i in range(self.T):
            self.YtXt[i] = np.outer(self.y[i], self.u_hat[i].T) # y[n] * E[x[n]].T
            self.YtYt[i] = np.outer(self.y[i], self.y[i].T) # y[n]y[n]
            self.XtYt[i] = np.outer(self.u_hat[i], self.y[i].T) #E[x[n]] * y[n].T 

        sub_1 = np.sum(self.XtXt_1[1:self.T], axis=0)
        sub_2 = np.sum(self.XtXt[0:self.T-1], axis=0)
        sub_3 = np.sum(self.XtXt[1:self.T], axis=0)
        sub_4 = np.sum(self.Xt_1Xt[1:self.T], axis=0)

        sub_5 = np.sum(self.YtXt, axis=0)
        sub_6 = np.sum(self.XtXt, axis=0)
        sub_7 = np.sum(self.YtYt, axis=0)
        sub_8 = np.sum(self.XtYt, axis=0)

        self.A = np.matmul(sub_1, np.linalg.inv(sub_2))

        self.Q = 1/(self.T-1) * (sub_3 - np.matmul(self.A,sub_4) - np.matmul(sub_1,self.A.T) + np.matmul(np.matmul(self.A,sub_2),self.A.T))
        
        self.C = np.matmul(sub_5, np.linalg.inv(sub_6))
        self.R = 1/self.T * (sub_7 - np.matmul(self.C,sub_8) - np.matmul(sub_5,self.C.T) + np.matmul(np.matmul(self.C,sub_6),self.C.T))


In [4]:
class SLDS(object):
    def __init__(self, K = None, T=None, transition_dis = None, init_prob = None, transition_con = None, lds_emission = None, hmm_emission = None, Q = None, R = None, u0 = None, V0 = None, y=None):

        if(transition_con is None or transition_dis is None or lds_emission is None):
            raise ValueError("Set proper system dynamics.")
        self.N = y.shape[1]
        self.M = transition_con.shape[1]
        self.T = T
        self.y = y
        
        self.C = lds_emission # the emission probability of the continuous variable, K x N x M
        self.A = transition_con # the continuous variable transition probability matrix, K x M x M 
        
        self.Q = Q # Q is the covariance matrix of noise term added to the hidden continuous state transition, K x M x M
        self.R = R # R is the covariance matrix of noise term added to the emission, K x N x N
        
        self.u0 = u0 # u0 is the initial estimate of the mean of x1, K x M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of x1, K x M x M
        
        self.hmm = HMM(K = K, T=T, transition_dis = transition_dis, init_prob = init_prob, emission = hmm_emission, y=[])
        emission_init = np.tile(np.mean(self.C,axis=0),(T,1)).reshape(T,self.N,self.M)
        transition_con_init = np.tile(np.mean(self.A,axis=0),(T,1)).reshape(T,self.M,self.M)
        Q_init = np.tile(np.mean(self.Q,axis=0),(T,1)).reshape(T,self.M,self.M) 
        R_init = np.tile(np.mean(self.R,axis=0),(T,1)).reshape(T,self.N,self.N)

        self.lds = timeVaryingLDS(T=T, transition_con = transition_con_init, emission = emission_init, Q = Q_init, R = R_init, u0 = np.mean(self.u0,axis=0), V0 = np.mean(self.V0,axis=0), y=y)


    def calculate_effective_con(self):
        K = self.hmm.K
        T = self.hmm.T
        N = self.N
        M = self.M

        R_hat = np.zeros((T,N,N))# T x N x N
        C_hat = np.zeros((T,N,M)) # T x N x M
        Q_hat = np.zeros((T, M, M)) # T x M x M
        A_hat = np.zeros((T,M,M)) # T x M x M
        V0_hat = np.zeros((M,M)) # M x M
        u0_hat = np.zeros(M) # M

        for t in range(T):
            R_hat_inv = np.zeros((N,N))
            for k in range(K):
                R_hat_inv += np.linalg.inv(self.R[k])*self.hmm.gamma[t,k]
                C_hat[t] += self.hmm.gamma[t,k]*np.linalg.inv(self.R[k]) @ self.C[k]
            R_hat[t] = np.linalg.inv(R_hat_inv)
            C_hat[t] = R_hat[t] @ C_hat[t]
        
        Q_hat_inv = np.zeros((M,M))
        for k in range(K):
            Q_hat_inv += self.hmm.gamma[-1,k]* np.linalg.inv(self.Q[k]) + self.hmm.gamma[-1,k]*self.C[k].T @ np.linalg.inv(self.R[k]) @ self.C[k]
        Q_hat_inv -= C_hat[-1].T @ np.linalg.inv(R_hat[-1]) @ C_hat[-1]
        Q_hat[-1] = np.linalg.inv(Q_hat_inv)
        
        for k in range(K):
            A_hat[-1] += self.hmm.gamma[-1,k] * np.linalg.inv(self.Q[k]) @ self.A[k]
        A_hat[-1] = Q_hat[-1] @ A_hat[-1]
        
        for t in range(T-2,-1,-1):
            Q_hat_inv = np.zeros((M,M))
            for k in range(K):
                Q_hat_inv += self.hmm.gamma[t,k]* np.linalg.inv(self.Q[k]) + self.hmm.gamma[t+1,k] * self.A[k].T @ np.linalg.inv(self.Q[k]) @ self.A[k] + self.hmm.gamma[t,k] * self.C[k].T @ np.linalg.inv(self.R[k]) @ self.C[k]
            Q_hat_inv -= A_hat[t+1].T @ np.linalg.inv(Q_hat[t+1]) @ A_hat[t+1] - C_hat[t].T @ np.linalg.inv(R_hat[t]) @ C_hat[t]
            Q_hat[t] = np.linalg.inv(Q_hat_inv)
            
            for k in range(K):
                A_hat[t] += self.hmm.gamma[t,k] * np.linalg.inv(self.Q[k]) @ self.A[k]
            A_hat[t] = Q_hat[t] @ A_hat[t]

        V0_hat_inv = np.zeros((M,M))
        for k in range(K):
            V0_hat_inv += self.hmm.gamma[0,k]* np.linalg.inv(self.V0[k]) + self.hmm.gamma[1,k] * self.A[k].T @ np.linalg.inv(self.Q[k]) @ self.A[k] + self.hmm.gamma[0,k] * self.C[k].T @ np.linalg.inv(self.R[k]) @ self.C[k]
        V0_hat_inv -= A_hat[1].T @ np.linalg.inv(Q_hat[1]) @ A_hat[1] - C_hat[0].T @ np.linalg.inv(R_hat[0]) @ C_hat[0]
        V0_hat = np.linalg.inv(V0_hat_inv)
        
        
        for k in range(K):
            u0_hat += self.hmm.gamma[0,k] * np.linalg.inv(self.V0[k]) @ self.u0[k]
        u0_hat = V0_hat @ u0_hat
        return R_hat, C_hat, Q_hat, A_hat, V0_hat, u0_hat
    
    def calculate_effective_emission(self):
        K = self.hmm.K
        T = self.hmm.T
        N = self.N
        emission_hat = np.zeros((T,K))
        for k in range(K):
            emission_hat[0,k] = -1/2*(np.trace(np.linalg.inv(self.V0[k]) @ self.lds.XtXt[0]) -2 * self.u0[k].T @ np.linalg.inv(self.V0[k]) @ self.lds.u_hat[0] + \
                                      self.u0[k].T @ np.linalg.inv(self.V0[k]) @ self.u0[k] + self.y[0].T @ np.linalg.inv(self.R[k]) @ self.y[0] - \
                                        2 * self.y[0].T @ np.linalg.inv(self.R[k]) @ self.C[k] @ self.lds.u_hat[0] + np.trace(self.C[k].T @ np.linalg.inv(self.R[k]) @ self.C[k] @ self.lds.XtXt[0]))- \
                                            1/2*np.log(np.linalg.det(self.V0[k]) )  
            emission_hat[0,k] = np.exp(emission_hat[0,k])    
        emission_hat[0,:] = emission_hat[0,:]/np.sum(emission_hat[0,:])  
        for t in range(1,T,1):
            for k in range(K):
                emission_hat[t,k] = -1/2*(np.trace(np.linalg.inv(self.Q[k])@ self.lds.XtXt[t]) -2 * np.trace(self.A[k].T @ np.linalg.inv(self.Q[k]) @ self.lds.XtXt_1[t]) + \
                                          np.trace(self.A[k].T @ np.linalg.inv(self.Q[k]) @ self.A[k] @ self.lds.XtXt[t-1]) + self.y[t].T @ np.linalg.inv(self.R[k]) @ self.y[t] - \
                                            2 * self.y[t].T @ np.linalg.inv(self.R[k]) @ self.C[k] @ self.lds.u_hat[t] + np.trace(self.C[k].T @ np.linalg.inv(self.R[k]) @ self.C[k] @ self.lds.XtXt[t]))- \
                                                1/2*np.log(np.linalg.det(self.R[k])) -1/2*np.log(np.linalg.det(self.Q[k]))
                emission_hat[t,k] = np.exp(emission_hat[t,k])
            emission_hat[t,:] = emission_hat[t,:]/np.sum(emission_hat[t,:])  
        print('hmm_emission is:',np.log(emission_hat[100,:]))
        return emission_hat
    
    def learning(self):
        K = self.hmm.K
        T = self.hmm.T
        for i in range(K):
            for j in range(K):
                self.hmm.transition[i,j] = np.sum(self.hmm.xi[0:-1,i,j])/np.sum(self.hmm.xi[0:-1,i,:])

        for i in range(K):
            self.hmm.init_prob[i] = self.hmm.gamma[0,i]/np.sum(self.hmm.gamma[0,:])

        for k in range(K):
            self.u0[k] = self.lds.u_hat[0]
            self.V0[k] = self.lds.XtXt[0] - np.outer(self.lds.u_hat[0],self.u0[k].T) - self.u0[k] @ self.lds.u_hat[0].T + np.outer(self.u0[k],self.u0[k].T)
            
            Ak_sub1 = np.zeros((self.M,self.M))
            Ak_sub2 = np.zeros((self.M,self.M))
            Qk_sub1 = np.zeros((self.M,self.M))
            Qk_sub2 = 0

            Ck_sub1 = np.zeros((self.N,self.M))
            Ck_sub2 = np.zeros((self.M,self.M))
            Rk_sub1 = np.zeros((self.N,self.N))
            Rk_sub2 = 0

            Ck_sub1 += self.hmm.gamma[0,k] * self.lds.YtXt[0]
            Ck_sub2 += self.hmm.gamma[0,k] * self.lds.XtXt[0]
            Rk_sub1 += self.hmm.gamma[0,k] * (self.lds.YtYt[0] - self.lds.YtXt[0]@self.C[k].T - self.C[k]@self.lds.XtYt[0] + self.C[k]@self.lds.XtXt[0]@self.C[k].T)
            Rk_sub2 += self.hmm.gamma[0,k]
            for t in range(1,self.T,1):
                Ak_sub1 += self.hmm.gamma[t,k] * self.lds.XtXt_1[t]
                Ak_sub2 += self.hmm.gamma[t,k] * self.lds.XtXt[t-1]
                Qk_sub1 += self.hmm.gamma[t,k] * (self.lds.XtXt[t] - self.lds.XtXt_1[t]@self.A[k].T - self.A[k]@self.lds.Xt_1Xt[t] + self.A[k]@self.lds.XtXt[t-1]@self.A[k].T)
                Qk_sub2 += self.hmm.gamma[t,k]

                Ck_sub1 += self.hmm.gamma[t,k] * self.lds.YtXt[t]
                Ck_sub2 += self.hmm.gamma[t,k] * self.lds.XtXt[t]
                Rk_sub1 += self.hmm.gamma[t,k] * (self.lds.YtYt[t] - self.lds.YtXt[t]@self.C[k].T - self.C[k]@self.lds.XtYt[t] + self.C[k]@self.lds.XtXt[t]@self.C[k].T)
                Rk_sub2 += self.hmm.gamma[t,k]
            self.A[k] = Ak_sub1 @ np.linalg.inv(Ak_sub2)
            self.Q[k] = Qk_sub1/Qk_sub2
            self.C[k] = Ck_sub1 @ np.linalg.inv(Ck_sub2)
            self.R[k] = Rk_sub1/Rk_sub2


In [5]:


# n_dis = 2 # Z
# n_con = 2 # M
# self.T = 2 # N
# n_time = 400 # T

# transition_dis = np.array([[0.95,0.05],[0.05,0.95]]) # the discrete variable transition probability matrix, Z x Z
# init_prob = np.array([0.4,0.6]) # the initial probability of the discrete variable, N

# emission = np.array([[[0.5, 0.5],[0.1, 0.9]],[[0.2, 0.8],[0.9, 0.1]]]) # the emission probability of the continuous variable, Z x N x M

# transition_con = np.array([[[0.9, 0.1],[0.1, 0.9]],[[0.1, 0.9],[0.1, 0.9]]]) # the continuous variable transition probability matrix, Z x M x M 

# Gamma = np.array([[[0.1, 0.3],[0.3, 0.1]],[[1, 0.5],[0.5, 1]]]) # Gamma is the covariance matrix of noise term added to the hidden state transition, Z x M x M
# Sigma = np.array([[0.2, 0.8],[0.8, 0.2]]) # Sigma is the covariance matrix of noise term added to the emission, N x N
# x0 = np.array([[0.2,0.2],[0.5, 0.5]]) # N x M



# states_dis, states_con,obs = generate_examples(T=n_time, Z = n_dis, M = n_con, N = self.T, transition_dis = transition_dis, init_prob = init_prob, 
# 											transition_con = transition_con, emission = emission, Gamma = Gamma, Sigma = Sigma, x0 = x0)


p_old = -10000
tol = 0.01
max_iter = 100



n_dis = 2               # Number of discrete latent states
n_obs = 2           # Observed data dimension
n_con = 2        # Latent state dimension
D_input = 0         # Exogenous input dimension
n_time = 2000            # Number of time steps to simulate
# K = 2               # Number of discrete latent states
# self.T = 2           # Observed data dimension
# n_con = 2        # Latent state dimension
# D_input = 0         # Exogenous input dimension
# T = 2000            # Number of time steps to simulate

true_mu_inits = [np.ones(n_con) for _ in range(n_dis)]
true_sigma_inits = [np.eye(n_con) for _ in range(n_dis)]
true_As = [.9 * random_rotation(n_con)
		for k in range(n_dis)]
true_Bs = [3 * npr.randn(n_con, D_input) for k in range(n_dis)]
true_sigma_states = [np.eye(n_con) for _ in range(n_dis)]
true_C = np.random.randn(n_obs, n_con)
true_Ds = np.zeros((n_obs, D_input))
true_sigma_obs = np.eye(n_obs)
print('true_mu\n',true_mu_inits,'\ntrue_sigma\n',true_sigma_inits,'\ntrue_As\n',true_As,'\ntrue_C\n',true_C,'\ntrue_sigma_states\n',true_sigma_states,'\ntrue_Ds\n',
	true_Ds,'\ntrue_sigma_obs\n',true_sigma_obs)

true_model = DefaultSLDS(n_dis, n_obs, n_con, D_input,mu_inits=true_mu_inits, sigma_inits=true_sigma_inits,
	As=true_As, Bs=true_Bs, sigma_statess=true_sigma_states,
	Cs=true_C, Ds=true_Ds, sigma_obss=true_sigma_obs)

inputs = npr.randn(n_time, D_input)
z = np.arange(n_dis).repeat(n_time // n_dis)

obs, states_con, states_dis = true_model.generate(n_time, inputs=inputs, stateseq=z)

transition_dis_init = np.array([[0.1,0.9],[0.2,0.8]]) 
init_prob_init = np.array([0.55,0.45]) 
emission_init = np.array([[[0.7, 0.3],[0.2, 0.8]],[[0.25, 0.75],[0.4, 0.6]]]) 
transition_con_init = np.array([[[0.9, 0.1],[0.1, 0.9]],[[0.1, 0.9],[0.1, 0.9]]])
Gamma_init = np.array([[[0.1, 0.2],[0.2, 0.1]],[[0.5, 0.3],[0.3, 0.5]]]) 
Sigma_init = np.array([[[0.2, 1.0],[1.0, 0.8]], [[1.0, 0],[0, 1.0]]])
u0 = np.array([[0.3, 0.3],[0.5, 0.5]]) # Z x M x 1
V0 = Gamma_init # Z x M x M

slds = SLDS(K = n_dis, T=n_time, transition_dis = transition_dis_init, init_prob = init_prob_init, transition_con = transition_con_init, 
			lds_emission = emission_init, hmm_emission = np.tile(emission_init[0,0],(n_time,1)).reshape(n_time,n_dis), Q = Gamma_init, R = Sigma_init, u0 = u0, V0 = V0, y=obs)
for ite in range(max_iter):

	# print(slds.forward,slds.backward)
	# print(slds.q,slds.h)
	# hidden_data = slds.hmm.generate_states()
	slds.hmm.forward_backward()
	slds.lds.kalman_filtering()
	slds.lds.kalman_smoothing()        
	slds.lds.kalman_learning()

	slds.learning()
	hmm_emission = slds.calculate_effective_emission()
	slds.hmm.emission = hmm_emission
	# print('\nhmm_emission\n',hmm_emission[10,:])
	R_hat, C_hat, Q_hat, A_hat, V0_hat, u0_hat = slds.calculate_effective_con()
	slds.lds.R = R_hat
	slds.lds.C = C_hat
	slds.lds.Q = Q_hat
	slds.lds.A = A_hat
	slds.lds.V0 = V0_hat
	slds.lds.u0 = u0_hat

	p = np.sum(np.log(slds.lds.c))
	print(f'The current iteration is: {ite}. The likelihood is {p}')
	if p>p_old and p - p_old < tol:
		break
	p_old = p

print('u0\n',slds.u0,'\nV0\n',slds.V0,'\ntransition_dis\n',slds.hmm.transition,'\ntransition_con\n',slds.A,'\nlds_emission\n',slds.C,'\nGamma\n',
	slds.Q,'\ninit_prob\n',slds.hmm.init_prob,'\nSigma\n',slds.Q,'\nhmm_emission\n',hmm_emission[10,:])




true_mu
 [array([1., 1.]), array([1., 1.])] 
true_sigma
 [array([[1., 0.],
       [0., 1.]]), array([[1., 0.],
       [0., 1.]])] 
true_As
 [array([[ 0.32935797,  0.83756989],
       [-0.83756989,  0.32935797]]), array([[ 0.77589174,  0.4560614 ],
       [-0.4560614 ,  0.77589174]])] 
true_C
 [[-0.07934559 -0.38174075]
 [-1.91615833 -0.98683022]] 
true_sigma_states
 [array([[1., 0.],
       [0., 1.]]), array([[1., 0.],
       [0., 1.]])] 
true_Ds
 [] 
true_sigma_obs
 [[1. 0.]
 [0. 1.]]
hmm_emission is: [-0.71981717 -0.66717004]
The current iteration is: 0. The likelihood is -43268.83843670958
hmm_emission is: [-0.71498654 -0.67177461]
The current iteration is: 1. The likelihood is -9383.427328016332
hmm_emission is: [-0.69431649 -0.69197924]
The current iteration is: 2. The likelihood is -9132.260631027837
hmm_emission is: [-0.69275897 -0.69353554]
The current iteration is: 3. The likelihood is -9130.44650249125
hmm_emission is: [-0.69359015 -0.69270441]
The current iteration is: 4. Th

In [6]:
test_model = DefaultSLDS(2*n_dis, n_obs, n_con, D_input,Cs=npr.randn(n_obs, n_con),
                         Ds=npr.randn(n_obs, D_input))
test_model.add_data(obs)

# Run the Gibbs sampler
N_samples = 1000
def update(model):
    model.resample_model()
    return model.log_likelihood()

lls = [update(test_model) for _ in range(N_samples)]

In [7]:
hmm_emission = slds.calculate_effective_emission()
slds.hmm.emission = hmm_emission
slds_states = slds.hmm.generate_states()

hmm_emission is: [-0.69383601 -0.69245882]


In [8]:
import sys
np.set_printoptions(threshold=sys.maxsize)


In [9]:
np.sum(np.abs(slds_states-states_dis))

836

In [10]:
slds.hmm.emission

array([[0.50000769, 0.49999231],
       [0.49927849, 0.50072151],
       [0.50022172, 0.49977828],
       [0.50028043, 0.49971957],
       [0.50013841, 0.49986159],
       [0.49970395, 0.50029605],
       [0.49953484, 0.50046516],
       [0.5002838 , 0.4997162 ],
       [0.50030198, 0.49969802],
       [0.49961351, 0.50038649],
       [0.50018537, 0.49981463],
       [0.50005489, 0.49994511],
       [0.50013652, 0.49986348],
       [0.50026542, 0.49973458],
       [0.49901135, 0.50098865],
       [0.50009536, 0.49990464],
       [0.5001524 , 0.4998476 ],
       [0.50004274, 0.49995726],
       [0.50027528, 0.49972472],
       [0.50017288, 0.49982712],
       [0.50020394, 0.49979606],
       [0.50012034, 0.49987966],
       [0.5000084 , 0.4999916 ],
       [0.49976179, 0.50023821],
       [0.50000905, 0.49999095],
       [0.4994674 , 0.5005326 ],
       [0.50025477, 0.49974523],
       [0.49992606, 0.50007394],
       [0.5001837 , 0.4998163 ],
       [0.50013344, 0.49986656],
       [0.

In [21]:
C_hat

array([[[ -34522.59651446,  139274.20725248],
        [-273050.58443052, 1050358.60259083]],

       [[ -34507.51016176,  139218.37439975],
        [-273175.67669787, 1050836.55099531]],

       [[ -34509.90880495,  139227.25161678],
        [-273155.7962463 , 1050760.5925494 ]],

       [[ -34509.54928299,  139225.92105295],
        [-273158.77624333, 1050771.97840769]],

       [[ -34509.61001555,  139226.1458197 ],
        [-273158.27284974, 1050770.05506085]],

       [[ -34509.59808723,  139226.10167386],
        [-273158.37172011, 1050770.43282093]],

       [[ -34509.58618474,  139226.05762364],
        [-273158.47037625, 1050770.80976254]],

       [[ -34509.58273491,  139226.04485606],
        [-273158.49897086, 1050770.91901571]],

       [[ -34509.60542466,  139226.12882916],
        [-273158.31090229, 1050770.20045059]],

       [[ -34509.60034569,  139226.11003224],
        [-273158.35300044, 1050770.36129757]],

       [[ -34509.58381831,  139226.04886565],
        [-2731

In [12]:
slds.hmm.gamma

array([[0.74937621, 0.25062379],
       [0.24061537, 0.75938463],
       [0.32143097, 0.67856903],
       [0.30931609, 0.69068391],
       [0.31136256, 0.68863744],
       [0.31096062, 0.68903938],
       [0.31055955, 0.68944045],
       [0.3104433 , 0.6895567 ],
       [0.31120786, 0.68879214],
       [0.31103672, 0.68896328],
       [0.3104798 , 0.6895202 ],
       [0.31105844, 0.68894156],
       [0.31080439, 0.68919561],
       [0.31110271, 0.68889729],
       [0.31104982, 0.68895018],
       [0.30994996, 0.69005004],
       [0.31107287, 0.68892713],
       [0.31091083, 0.68908917],
       [0.31085234, 0.68914766],
       [0.31105749, 0.68894251],
       [0.31094601, 0.68905399],
       [0.31100079, 0.68899921],
       [0.31096342, 0.68903658],
       [0.31083138, 0.68916862],
       [0.31072634, 0.68927366],
       [0.31085368, 0.68914632],
       [0.31040183, 0.68959817],
       [0.31111995, 0.68888005],
       [0.31072665, 0.68927335],
       [0.31099711, 0.68900289],
       [0.